In [56]:
#Équipe 7
#Membres de l'équipe 
#Foulémata Tirera, Gabriel Laberge, Morgan Péju, Maninder Bhogal, Jean-Christophe Desmarais, Pascal Dally-Bélanger


In [57]:
import pandas as pd
from functools import reduce
film_df = pd.read_csv("donnees/film.csv")
#keeping columns that have no missing values
film_df = film_df.rename(columns={"FilmoId": "filmoId"}, errors="raise")
film_df = film_df[['filmoId', 'titreOriginal', 'anneeSortie']]
film_df
film_df.to_csv(r'data/film_year.csv', index = False, header=True)


In [58]:
#donnees de wikidata

film_genres = pd.read_csv("donnees/movie_genres.csv")
film_langues = pd.read_csv("donnees/movie_langues.csv")
film_places = pd.read_csv("donnees/movie_places.csv")
film_directors = pd.read_csv("donnees/movie_directors.csv")


In [59]:
#donnees de cinématèque quebecoise 
#*********directors cleanup ***************

#combining the names of the actors with the movies they were in
nom = pd.read_csv("donnees/Nom.csv")
film_realisation = pd.read_csv("donnees/Filmo_Realisation.csv")
film_realisation['NomId'] = film_realisation['nomId']
film_director = pd.merge(film_realisation,nom, on='NomId', how="inner")


#combining the first name and last name of the actor
#film_director = film_director[['filmoId','nom','prenom']]
film_director['Nom']= film_director['prenom'] + ' ' + film_director['nom']
film_director = film_director.drop(columns=["nom", "prenom"])

#getting functions of each person
fonction = pd.read_csv("donnees/Fonction.csv")
filmo_generique = pd.read_csv("donnees/Filmo_Generique.csv")
filmo_generique['FonctionId'] = filmo_generique['fonctionId']
film_fonctions = pd.merge(fonction,filmo_generique, on='FonctionId', how="inner")
film_fonctions['Role'] = film_fonctions['terme']
film_fonctions = film_fonctions.drop(columns=["filmoId","terme", "organismeId", "fonctionId", "FonctionId","GeneriqueId"])

#PERSONNES ET GENRES
film_fonctions = pd.merge(film_fonctions,film_director,on="nomId", how="inner")
film_fonctions = film_fonctions[['Nom','Role','filmoId']]

film_fonctions.to_csv(r'data/film_personRole.csv', index = False, header=True)


In [60]:
#donnees de cinématèque quebecoise 
#*********langue cleanup ***************

langue = pd.read_csv("donnees/Langue.csv")
filmo_langue = pd.read_csv("donnees/Filmo_Langue.csv")
filmo_langue['LangueId'] = filmo_langue['langueId']
film_langue = pd.merge(filmo_langue,langue, on='LangueId', how="inner")
film_langue['langue'] = film_langue[['terme']]
film_langue = film_langue[['filmoId','langue']]

film_langue.to_csv(r'data/film_langue.csv', index = False, header=True)


In [61]:
#donnees de cinématèque quebecoise 
#*********pays cleanup ***************

pays = pd.read_csv("donnees/Pays.csv")
filmo_pays = pd.read_csv("donnees/Filmo_Pays.csv")
filmo_pays['PaysId'] = filmo_pays['paysId']
film_pays = pd.merge(filmo_pays,pays, on='PaysId', how="inner")
film_pays['pays'] = film_pays[['terme']]
film_pays = film_pays[['filmoId','pays']]

film_pays.to_csv(r'data/film_pays.csv', index = False, header=True)


In [62]:
#donnees de cinématèque quebecoise 
#LANGUE ET PAYS 
dfs = [film_langue, film_pays]
langue_pays = reduce(lambda left, right: pd.merge(left, right, on='filmoId', how="inner"), dfs)
langue_pays

#there's more colummns now because some movies are produced in two countries 

,filmoId,langue,pays
0,5,anglais,États-Unis
1,25,anglais,Italie
2,25,anglais,Royaume-Uni
3,25,français,Italie
4,25,français,Royaume-Uni
...,...,...,...
52300,103925,Dzongkha,Finlande
52301,103925,Dzongkha,Bhoutan
52302,105511,Dzongkha,Espagne
52303,105511,Dzongkha,Cuba


In [63]:
#donnees de cinématèque quebecoise 
#*********genre cleanup ***************

genres = pd.read_csv("donnees/GenresCategories_Wikidata.csv")
filmo_genre = pd.read_csv("donnees/Filmo_GenreCategories.csv")
filmo_genre = pd.merge(filmo_genre,genres, on='sujetId', how="inner")
filmo_genre['genre'] = filmo_genre['lienWikidata']
filmo_genre = filmo_genre[['filmoId','genre']]
filmo_genre

,filmoId,genre
0,57864,http://www.wikidata.org/entity/Q93204
1,55784,http://www.wikidata.org/entity/Q93204
2,20937,http://www.wikidata.org/entity/Q93204
3,53108,http://www.wikidata.org/entity/Q93204
4,61024,http://www.wikidata.org/entity/Q93204
...,...,...
121030,62691,http://www.wikidata.org/entity/Q229390
121031,75863,http://www.wikidata.org/entity/Q229390
121032,97606,http://www.wikidata.org/entity/Q229390
121033,109769,http://www.wikidata.org/entity/Q229390


In [64]:
#getting movies genre label   
movies_genre = pd.merge(film_df, filmo_genre, on='filmoId', how="inner")
film_genres = film_genres.drop(columns=['film','titreOriginal'])
#movies = movies.sort_values(by='filmoId')
film_genres = film_genres.drop_duplicates()
#film_genres
genre_labels = pd.merge(filmo_genre, film_genres, on='genre', how="inner")
genre_labels

,filmoId,genre,genreLabel
0,57864,http://www.wikidata.org/entity/Q93204,SÉRIES DOCUMENTAIRES (TV)
1,57864,http://www.wikidata.org/entity/Q93204,SÉRIES DOCUMENTAIRES
2,57864,http://www.wikidata.org/entity/Q93204,DOCUMENTAIRES (VIDÉO)
3,57864,http://www.wikidata.org/entity/Q93204,DOCUMENTAIRE
4,57864,http://www.wikidata.org/entity/Q93204,SÉRIE DOCUMENTAIRE
...,...,...,...
372993,107830,http://www.wikidata.org/entity/Q226730,MUET
372994,107831,http://www.wikidata.org/entity/Q226730,MUET
372995,108164,http://www.wikidata.org/entity/Q226730,MUET
372996,436,http://www.wikidata.org/entity/Q226730,MUET


In [65]:
#genre combiend with data from database

movies_genre = pd.merge(film_df, genre_labels, on='filmoId', how="inner")
movies_genre = movies_genre.sort_values(by='filmoId')
movies_genre.to_csv(r'data/film_genres.csv', index = False, header=True)
#movies_genre.groupby(movies_genre['filmoId']).count()

In [66]:
#genre combined with data from wikidata 

#movies_genre_wiki = pd.merge(film_df,film_genres,on="titreOriginal",how="inner")
#movies_genre_wiki
#movies_genre_wiki.groupby(movies_genre_wiki['filmoId']).count()

In [67]:
film_fonctions = pd.merge(movies_genre,film_fonctions,on="filmoId")
film_direction = film_fonctions[film_fonctions['Role']=="Direction de production"]
#film_direction = film_direction.drop(columns=["nomId","NomId"])
film_direction

,filmoId,titreOriginal,anneeSortie,genre,genreLabel,Nom,Role
5201,252,PASSIFLORA,1985.0,http://www.wikidata.org/entity/Q93204,DOCUMENTAIRES,Dagmar Gueissaz-Teufel,Direction de production
5245,252,PASSIFLORA,1985.0,http://www.wikidata.org/entity/Q93204,DOCUMENTAIRES (TV),Dagmar Gueissaz-Teufel,Direction de production
5289,252,PASSIFLORA,1985.0,http://www.wikidata.org/entity/Q93204,SÉRIE DOCUMENTAIRE,Dagmar Gueissaz-Teufel,Direction de production
5333,252,PASSIFLORA,1985.0,http://www.wikidata.org/entity/Q93204,DOCUMENTAIRES (VIDÉO),Dagmar Gueissaz-Teufel,Direction de production
5377,252,PASSIFLORA,1985.0,http://www.wikidata.org/entity/Q93204,SÉRIES DOCUMENTAIRES,Dagmar Gueissaz-Teufel,Direction de production
...,...,...,...,...,...,...,...
4686892,109928,COMPAGNONS DU LION D'OR,NaN,http://www.wikidata.org/entity/Q93204,SÉRIES DOCUMENTAIRES (TV),Louis Ricard,Direction de production
4686903,109928,COMPAGNONS DU LION D'OR,NaN,http://www.wikidata.org/entity/Q93204,DOCUMENTAIRES (TV),Louis Ricard,Direction de production
4690212,110007,COULEUR DE TES LÈVRES,2018.0,http://www.wikidata.org/entity/Q459290,"INDÉPENDANTE, VIDÉO",Annick Blanc,Direction de production
4690226,110007,COULEUR DE TES LÈVRES,2018.0,http://www.wikidata.org/entity/Q459290,PRODUCTION INDÉPENDANTE,Annick Blanc,Direction de production


In [68]:
film_directors = pd.merge(movies_genre,film_directors,on="titreOriginal")
#film_direction = film_direction.drop(columns=["nomId","NomId"])
film_directors

,filmoId,titreOriginal,anneeSortie,genre,genreLabel,film,director,directorLabel
0,5,GUILTY BY SUSPICION,1991.0,http://www.wikidata.org/entity/Q2484376,THRILLERS,http://data.cinematheque.qc.ca/resource/Work5,http://www.wikidata.org/entity/Q950428,Irwin Winkler
1,5,GUILTY BY SUSPICION,1991.0,http://www.wikidata.org/entity/Q2484376,THRILLERS (TV),http://data.cinematheque.qc.ca/resource/Work5,http://www.wikidata.org/entity/Q950428,Irwin Winkler
2,5,GUILTY BY SUSPICION,1991.0,http://www.wikidata.org/entity/Q2484376,THRILLERS (VIDÉO),http://data.cinematheque.qc.ca/resource/Work5,http://www.wikidata.org/entity/Q950428,Irwin Winkler
3,5,GUILTY BY SUSPICION,1991.0,http://www.wikidata.org/entity/Q2484376,THRILLER,http://data.cinematheque.qc.ca/resource/Work5,http://www.wikidata.org/entity/Q950428,Irwin Winkler
4,17,URANUS,1990.0,http://www.wikidata.org/entity/Q157443,COMÉDIES (VIDÉO),http://data.cinematheque.qc.ca/resource/Work17,http://www.wikidata.org/entity/Q298905,Claude Berri
...,...,...,...,...,...,...,...,...
124110,110070,BROKEN SILENCE,NaN,http://www.wikidata.org/entity/Q93204,DOCUMENTAIRES,http://data.cinematheque.qc.ca/resource/Work11...,http://www.wikidata.org/entity/Q1374327,Vojtech Jasny
124111,110070,BROKEN SILENCE,NaN,http://www.wikidata.org/entity/Q93204,DOCUMENTAIRES,http://data.cinematheque.qc.ca/resource/Work11...,http://www.wikidata.org/entity/Q611429,Luis Puenzo
124112,110070,BROKEN SILENCE,NaN,http://www.wikidata.org/entity/Q93204,DOCUMENTAIRES (TV),http://data.cinematheque.qc.ca/resource/Work11...,http://www.wikidata.org/entity/Q106685,Andrzej Wajda
124113,110070,BROKEN SILENCE,NaN,http://www.wikidata.org/entity/Q93204,DOCUMENTAIRES (TV),http://data.cinematheque.qc.ca/resource/Work11...,http://www.wikidata.org/entity/Q1374327,Vojtech Jasny


In [69]:
film_directors.to_csv(r'data/film_directors_wiki.csv', index = False, header=True)

In [70]:
film_direction.to_csv(r'data/film_directors_cmtq.csv', index = False, header=True)